In [235]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

#Lets get weather data
#source: https://forecast.weather.gov

In [236]:
weather_page = requests.get("https://forecast.weather.gov/MapClick.php?lat=40.7175&lon=-74.0438")
soup = BeautifulSoup(weather_page.content, 'html.parser')
#print(soup.prettify())

In [237]:
seven_day = soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_="tombstone-container")

##Use this *today* template to create 7 day forecast
today_info = forecast_items[0]
day = today_info.find(class_='period-name').get_text()
short_desc = today_info.find(class_='short-desc').get_text()
temp = today_info.find(class_='temp temp-high').get_text()

print(day, short_desc, temp, sep='\n')

#Note: will have to handle proper spacing in description section

Today
ChanceShowers
High: 73 °F


In [239]:
title_tag = seven_day.select('img')
titles = [i['title'] for i in title_tag]

titles

['Today: A 50 percent chance of showers.  Cloudy, with a high near 73. Southeast wind 7 to 9 mph. ',
 'Tonight: Showers, mainly after 2am.  Low around 65. East wind 6 to 8 mph.  Chance of precipitation is 80%. New precipitation amounts between a quarter and half of an inch possible. ',
 'Saturday: Showers and possibly a thunderstorm. Some of the storms could produce heavy rain.  High near 70. East wind 5 to 11 mph becoming northwest in the afternoon.  Chance of precipitation is 80%. New rainfall amounts between three quarters and one inch possible. ',
 'Saturday Night: Showers likely, mainly before 8pm.  Mostly cloudy during the early evening, then gradual clearing, with a low around 50. Northwest wind 13 to 15 mph.  Chance of precipitation is 60%. New precipitation amounts of less than a tenth of an inch possible. ',
 'Sunday: Increasing clouds, with a high near 62. West wind 15 to 18 mph. ',
 'Sunday Night: Partly cloudy, with a low around 48.',
 'Columbus Day: Mostly sunny, with a h

In [240]:
#locate 7 day forecast
seven_day = soup.find(id="seven-day-forecast")

grab_days = seven_day.select(".tombstone-container .period-name")

day = [i.text for i in grab_days]

#Have to create a string for the regex
string_day =' '.join(day)

import re

#look for a pattern of lowercase letter followed by uppercase letter and put in a separator
#but nothing too complicated since it will need to be split on whitespace to put back into
# a list

p = r'([a-z])([A-Z])'
 
re_days = re.sub(p,r'\1_\2',string_day)

#Put back into a list
day_list = re_days.split()
day_list

['Today',
 'Tonight',
 'Saturday',
 'Saturday_Night',
 'Sunday',
 'Sunday_Night',
 'Columbus_Day',
 'Monday_Night',
 'Tuesday']

In [241]:
#Grab the descriptive text

grab_desc = seven_day.select(".tombstone-container .short-desc")
desc = [i.text for i in grab_desc]


#apply same regex as before

re2 = r'([a-z])(and|then)'

descrip = []
for i in desc:
    re_desc = re.sub(p,r'\1 \2',i) #Fix spacing --> SlightChanceShowers
    re_desc2 = re.sub(re2,r'\1 \2',re_desc) #Had to add another to handle 'Cloudythen'
    descrip.append(re_desc2)
descrip

['Chance Showers',
 'Chance Showers then Showers',
 'Heavy Rain',
 'Showers Likely then Slight Chance Showers',
 'Increasing Clouds',
 'Partly Cloudy',
 'Mostly Sunny',
 'Mostly Cloudy then Chance Showers',
 'Chance Showers then Partly Sunny']

In [242]:
grab_temp = seven_day.select(".tombstone-container .temp")
temps = [i.text for i in grab_temp]

temps

['High: 73 °F',
 'Low: 65 °F',
 'High: 70 °F',
 'Low: 50 °F',
 'High: 62 °F',
 'Low: 48 °F',
 'High: 60 °F',
 'Low: 51 °F',
 'High: 66 °F']

In [243]:
#Create a weather dataframe

weather = pd.DataFrame(columns=['Time_of_day','Feels_like','Temperature','Full_description'])

In [244]:
#Populate the dataframe with the lists previously generated

weather['Temperature']= [i for i in temps]
weather['Feels_like']= [i for i in descrip]
weather['Time_of_day']= [i for i in day_list]
weather['Full_description']= [i for i in titles]

In [245]:
#expand the column widths so that all of the text shows

pd.options.display.max_colwidth=150

In [246]:
weather

,Time_of_day,Feels_like,Temperature,Full_description
0,Today,Chance Showers,High: 73 °F,"Today: A 50 percent chance of showers. Cloudy, with a high near 73. Southeast wind 7 to 9 mph."
1,Tonight,Chance Showers then Showers,Low: 65 °F,"Tonight: Showers, mainly after 2am. Low around 65. East wind 6 to 8 mph. Chance of precipitation is 80%. New precipitation amounts between a qua..."
2,Saturday,Heavy Rain,High: 70 °F,Saturday: Showers and possibly a thunderstorm. Some of the storms could produce heavy rain. High near 70. East wind 5 to 11 mph becoming northwes...
3,Saturday_Night,Showers Likely then Slight Chance Showers,Low: 50 °F,"Saturday Night: Showers likely, mainly before 8pm. Mostly cloudy during the early evening, then gradual clearing, with a low around 50. Northwest..."
4,Sunday,Increasing Clouds,High: 62 °F,"Sunday: Increasing clouds, with a high near 62. West wind 15 to 18 mph."
5,Sunday_Night,Partly Cloudy,Low: 48 °F,"Sunday Night: Partly cloudy, with a low around 48."
6,Columbus_Day,Mostly Sunny,High: 60 °F,"Columbus Day: Mostly sunny, with a high near 60."
7,Monday_Night,Mostly Cloudy then Chance Showers,Low: 51 °F,"Monday Night: A 30 percent chance of showers after 2am. Mostly cloudy, with a low around 51."
8,Tuesday,Chance Showers then Partly Sunny,High: 66 °F,"Tuesday: A 30 percent chance of showers before 8am. Partly sunny, with a high near 66."
